### Requirements

In [ ]:
!pip install torchvision -q

### Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler

from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
import itertools
import numpy as np
import time

### Import ResNet Functions

In [2]:
try:
    from ResNet_pytorch import resnet18
    print("Successfully imported ResNet_pytorch.py")
except ImportError:
    print("Error: Could not import ResNet_pytorch.py.")
    print("Please make sure 'ResNet_pytorch.py' is in the same directory.")
    exit()

Successfully imported ResNet_pytorch.py


### SetUp

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


### Hyperparameter

In [4]:
LEARNING_RATE = 1e-5
BATCH_SIZE = 32
NUM_CLASSES = 10      # FashionMNIST has 10 classes
INPUT_CHANNELS = 1    # FashionMNIST is grayscale
NUM_EPOCHS = 80

### Normalization

In [5]:
image_transforms = transforms.Compose([
    transforms.Resize(224), # ResNet-18 standard input size
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5], # Grayscale mean
        std=[0.5]   # Grayscale std
    ),
])

### Load Dataset

In [6]:
# Training
train_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=image_transforms
)
# Testing
test_dataset = torchvision.datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=image_transforms
)

### Data Loader

In [7]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=5
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=5
)

### Model Initialization

In [8]:
model = resnet18(num_classes=NUM_CLASSES)

model.conv1 = nn.Conv2d(INPUT_CHANNELS, 64, kernel_size=3, stride=1, padding=1, bias=False) # ResNet-9
model.maxpool = nn.Identity()

model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.AdamW(model.parameters(),lr=LEARNING_RATE,weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=NUM_EPOCHS, gamma=0.1)


### Model Setup

In [9]:
def train_model(model, train_loader, criterion, optimizer, num_epochs, device):
    """Handles the training process and evaluates after each epoch, tracking losses."""
    model.train()
    print("\nStarting training on FashionMNIST...")

    # Clear logs before starting
    train_losses, test_losses, test_accuracies=[],[],[]

    for epoch in range(num_epochs):
        total_loss = 0.0
        total_samples = 0

        # Training Loop
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # === sample-weighted loss, matches evaluation ===
            batch_size = labels.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size

        # === Correct per-sample training loss ===
        epoch_loss = total_loss / total_samples
        train_losses.append(epoch_loss)

        # Evaluate on test set
        acc, test_loss = evaluate_model(model, test_loader, criterion, device)

        test_losses.append(test_loss)
        test_accuracies.append(acc)

        print(f"Epoch {epoch+1} finished. Train Loss: {epoch_loss:.4f} | Test Loss: {test_loss:.4f} | Test Accuracy: {acc:.2f}%")
    return train_losses, test_losses, test_accuracies


def evaluate_model(model, test_loader, criterion, device):
    """Calculates the Top-1 accuracy AND the average test loss."""
    model.eval()
    correct_predictions = 0
    total_samples = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            batch_size = labels.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size

            _, predicted = outputs.max(1)
            correct_predictions += (predicted == labels).sum().item()

    accuracy = 100 * correct_predictions / total_samples
    avg_loss = total_loss / total_samples

    return accuracy, avg_loss


### Model Training

In [10]:
import gc
gc.collect()
torch.cuda.empty_cache()
train_losses, test_losses, test_accuracies = train_model(
    model, train_loader, criterion, optimizer, NUM_EPOCHS, DEVICE
)


Starting training on FashionMNIST...


Epoch 1/80: 100%|██████████| 1875/1875 [08:31<00:00,  3.66it/s]


Epoch 1 finished. Train Loss: 0.8406 | Test Loss: 0.6289 | Test Accuracy: 78.94%


Epoch 2/80: 100%|██████████| 1875/1875 [08:24<00:00,  3.71it/s]


Epoch 2 finished. Train Loss: 0.5047 | Test Loss: 0.5189 | Test Accuracy: 81.86%


...

Epoch 78 finished. Train Loss: 0.0280 | Test Loss: 0.3083 | Test Accuracy: 93.02%


Epoch 79/80: 100%|██████████| 1875/1875 [04:10<00:00,  7.48it/s]


Epoch 79 finished. Train Loss: 0.0282 | Test Loss: 0.3264 | Test Accuracy: 92.97%


Epoch 80/80: 100%|██████████| 1875/1875 [04:10<00:00,  7.48it/s]


Epoch 80 finished. Train Loss: 0.0257 | Test Loss: 0.3455 | Test Accuracy: 92.61%


In [35]:
import pandas as pd
def save_metrics_to_csv(csv_path="resnet18_mnist.csv"):
    df = pd.DataFrame({
        "epoch": list(range(1, len(train_losses) + 1)),
        "train_loss": train_losses,
        "test_loss": test_losses,
        "test_accuracy": test_accuracies,
    })
    df.to_csv(csv_path, index=False)
    print(f"Saved training log to {csv_path}")
save_metrics_to_csv("resnet18_mnist.csv")

Saved training log to resnet18_mnist.csv


In [13]:
import pandas as pd
df = pd.DataFrame({
        "epoch": list(range(1, len(train_losses) + 1)),
        "train_loss": train_losses,
        "test_loss": test_losses,
        "test_accuracy": test_accuracies,
    })
df.to_csv("resnet18_mnist.csv")

In [12]:
! pip install pandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 16.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 36.6 MB/s eta 0:00:00
